# Construir un Asistente LLM basado en RAG usando Streamlit y Snowflake Cortex Search

*NOTA: Para requisitos previos y otras instrucciones, consulta la [Guía de Inicio Rápido](https://quickstarts.snowflake.com/guide/ask_questions_to_your_own_documents_with_snowflake_cortex_search/index.html#0).*


## Configuración Inicial

Crear una base de datos y un schema.


In [ ]:
--CREATE DATABASE IF NOT EXISTS CC_QUICKSTART_CORTEX_SEARCH_DOCS;
--CREATE SCHEMA IF NOT EXISTS DATA;


## Organizar Documentos y Crear Función de Pre-procesamiento

Paso 1. Descargar [documentos PDF de muestra](https://github.com/sfc-gh-jparrado/HOL/tree/main/Cortex_Search/documentos).


Paso 2. Crear un Stage con Directory Table donde subirás tus documentos.


In [ ]:
create or replace stage docs ENCRYPTION = (TYPE = 'SNOWFLAKE_SSE') DIRECTORY = ( ENABLE = true );


Paso 3. Subir documentos a tu área de staging

- Selecciona Data en el menú izquierdo
- Haz clic en tu base de datos CC_QUICKSTART_CORTEX_SEARCH_DOCS
- Haz clic en tu schema DATA
- Haz clic en Stages y selecciona DOCS
- En la esquina superior derecha haz clic en el botón **+Files**
- Arrastra y suelta los documentos PDF que descargaste


Paso 4. Verificar que los archivos se han subido correctamente


In [ ]:
ls @docs;


## Pre-procesar y Etiquetar Documentos

Paso 1. Crear la tabla donde vamos a almacenar los chunks para cada PDF.

Vamos a aprovechar las funciones nativas de procesamiento de documentos de Snowflake para preparar los documentos antes de habilitar Cortex Search. También vamos a usar la función Cortex CLASSIFY_TEXT para etiquetar el tipo de documento que se está procesando, de modo que podamos usar esos metadatos para filtrar búsquedas.


In [ ]:
CREATE OR REPLACE TEMPORARY TABLE RAW_TEXT AS
SELECT 
    RELATIVE_PATH,
    SIZE,
    FILE_URL,
    build_scoped_file_url(@docs, relative_path) as scoped_file_url,
    TO_VARCHAR (
        SNOWFLAKE.CORTEX.PARSE_DOCUMENT (
            '@docs',
            RELATIVE_PATH,
            {'mode': 'LAYOUT'} ):content
        ) AS EXTRACTED_LAYOUT 
FROM 
    DIRECTORY('@docs');


In [ ]:
create or replace TABLE DOCS_CHUNKS_TABLE ( 
    RELATIVE_PATH VARCHAR(16777216), -- Ruta relativa al archivo PDF
    SIZE NUMBER(38,0), -- Tamaño del PDF
    FILE_URL VARCHAR(16777216), -- URL para el PDF
    SCOPED_FILE_URL VARCHAR(16777216), -- URL con scope (puedes elegir cuál mantener según tu caso de uso)
    CHUNK VARCHAR(16777216), -- Fragmento de texto
    CHUNK_INDEX INTEGER, -- Índice del texto
    CATEGORY VARCHAR(16777216) -- Contendrá la categoría del documento para habilitar filtrado
);


Paso 2. Usar la función CORTEX PARSE_DOCUMENT para leer los documentos PDF del área de staging. No es necesario crear embeddings ya que serán gestionados automáticamente por el servicio Cortex Search más adelante.


In [ ]:
 insert into docs_chunks_table (relative_path, size, file_url,
                            scoped_file_url, chunk, chunk_index)

    select relative_path, 
            size,
            file_url, 
            scoped_file_url,
            c.value::TEXT as chunk,
            c.INDEX::INTEGER as chunk_index
            
    from 
        raw_text,
        LATERAL FLATTEN( input => SNOWFLAKE.CORTEX.SPLIT_TEXT_RECURSIVE_CHARACTER (
              EXTRACTED_LAYOUT,
              'markdown',
              1512,
              256,
              ['\n\n', '\n', ' ', '']
           )) c;


### Etiquetar la categoría del producto

Vamos a usar el poder de los Modelos de Lenguaje Grande y la función [CLASSIFY_TEXT](https://docs.snowflake.com/en/sql-reference/functions/classify_text-snowflake-cortex) para clasificar fácilmente los documentos que estamos ingiriendo en nuestra aplicación RAG. Vamos a pasar el nombre del documento y el primer chunk de texto a la función classify_text.

Primero crearemos una tabla temporal con cada nombre de archivo único y pasaremos ese nombre de archivo y el primer chunk de texto a CLASSIFY_TEXT. La clasificación no es obligatoria para Cortex Search, pero queremos usarla aquí para también demostrar la búsqueda híbrida.

Ejecuta este SQL para crear esa tabla:


In [ ]:
CREATE OR REPLACE TEMPORARY TABLE docs_categories AS WITH unique_documents AS (
  SELECT
    DISTINCT relative_path, chunk
  FROM
    docs_chunks_table
  WHERE 
    chunk_index = 0
  ),
 docs_category_cte AS (
  SELECT
    relative_path,
    TRIM(snowflake.cortex.CLASSIFY_TEXT (
      'Title:' || relative_path || 'Content:' || chunk, ['Bike', 'Snow']
     )['label'], '"') AS category
  FROM
    unique_documents
)
SELECT
  *
FROM
  docs_category_cte;


Puedes revisar esa tabla para identificar cuántas categorías se han creado y si son correctas:


In [ ]:
select category from docs_categories group by category;


También podemos verificar que la categoría de cada documento sea correcta:


In [ ]:
select * from docs_categories;


Ahora podemos actualizar la tabla con los chunks de texto que serán usados por el servicio Cortex Search para incluir la categoría de cada documento:


In [ ]:
update docs_chunks_table 
  SET category = docs_categories.category
  from docs_categories
  where  docs_chunks_table.relative_path = docs_categories.relative_path;


## Crear Servicio Cortex Search

El siguiente paso es crear el SERVICIO CORTEX SEARCH en la tabla que creamos antes.

- El nombre del servicio es CC_SEARCH_SERVICE_CS.
- El servicio usará la columna chunk para crear embeddings y realizar recuperación basada en búsqueda de similitud.
- La columna category podrá ser usada como filtro.
- Para mantener este servicio actualizado, se usará el warehouse COMPUTE_WH. NOTA: Puedes reemplazar el nombre del warehouse con otro al que tengas acceso.
- El servicio se refrescará cada minuto.
- Los datos recuperados contendrán el chunk, relative_path, file_url y category.


In [ ]:
create or replace CORTEX SEARCH SERVICE CC_SEARCH_SERVICE_CS
ON chunk
ATTRIBUTES category
warehouse = COMPUTE_WH
TARGET_LAG = '1 minute'
as (
    select chunk,
        chunk_index,
        relative_path,
        file_url,
        category
    from docs_chunks_table
);


## Construir Interfaz de Chat

Para construir y ejecutar la interfaz de chat en Streamlit, continúa y completa los pasos descritos en la [Guía de Inicio Rápido](https://quickstarts.snowflake.com/guide/ask_questions_to_your_own_documents_with_snowflake_cortex_search/index.html#4).




## Procesamiento Automático de Nuevos Documentos
Mantener tu sistema RAG actualizado cuando se agregan, eliminan o actualizan nuevos documentos puede ser tedioso. Snowflake lo hace muy fácil. Por un lado, Cortex Search es un servicio auto-gestionado. Solo necesitamos agregar, eliminar o actualizar filas en la tabla donde se ha habilitado el servicio Cortex Search y automáticamente el servicio actualizará los índices y creará nuevos embeddings basándose en la frecuencia definida durante la creación del servicio.

Además, podemos usar características de Snowflake como Streams, Tasks y Stored Procedures para procesar automáticamente nuevos archivos PDF a medida que se agregan a Snowflake. 

Primero creamos dos streams para el área de staging DOCS. Uno se usará para procesar eliminaciones y otro para procesar inserciones. Los Streams capturan los cambios en la Directory Table usada para el área de staging DOCS. Así podemos rastrear nuevas actualizaciones y eliminaciones:


In [ ]:
create or replace stream insert_docs_stream on stage docs;
create or replace stream delete_docs_stream on stage docs;


Segundo, vamos a definir un Stored Procedure que procese esos streams para:

- Eliminar de la tabla docs_chunk_table el contenido de archivos que han sido eliminados del área de staging, para que ya no sean relevantes
- Analizar nuevos documentos PDF que han sido agregados al área de staging usando PARSE_DOCUMENT
- Fragmentar el nuevo documento en piezas usando SPLIT_TEXT_RECURSIVE_CHARACTER
- Clasificar los nuevos documentos y actualizar la etiqueta (este paso es opcional, solo para mostrar lo que es posible)

Crear el Stored Procedure:


In [ ]:
create or replace procedure insert_delete_docs_sp()
RETURNS VARCHAR
LANGUAGE SQL
AS
$$
BEGIN

DELETE FROM docs_chunks_table
    USING delete_docs_stream
    WHERE docs_chunks_table.RELATIVE_PATH = delete_docs_stream.RELATIVE_PATH
    and delete_docs_stream.METADATA$ACTION = 'DELETE';


CREATE or replace TEMPORARY table RAW_TEXT AS
    SELECT 
        RELATIVE_PATH,
        SIZE,
        FILE_URL,
        build_scoped_file_url(@docs, relative_path) as scoped_file_url,
        TO_VARCHAR (
            SNOWFLAKE.CORTEX.PARSE_DOCUMENT (
                '@docs',
                RELATIVE_PATH,
                {'mode': 'LAYOUT'} ):content
            ) AS EXTRACTED_LAYOUT 
    FROM 
        insert_docs_stream
    WHERE 
        METADATA$ACTION = 'INSERT';

    -- Insert new docs chunks
    insert into docs_chunks_table (relative_path, size, file_url,
                            scoped_file_url, chunk, chunk_index)

    select relative_path, 
            size,
            file_url, 
            scoped_file_url,
            c.value::TEXT as chunk,
            c.INDEX::INTEGER as chunk_index
            
    from 
        RAW_TEXT,
        LATERAL FLATTEN( input => SNOWFLAKE.CORTEX.SPLIT_TEXT_RECURSIVE_CHARACTER (
              EXTRACTED_LAYOUT,
              'markdown',
              1512,
              256,
              ['\n\n', '\n', ' ', '']
           )) c;

    -- Classify the new documents

    CREATE OR REPLACE TEMPORARY TABLE docs_categories AS 
    WITH unique_documents AS (
      SELECT DISTINCT
        d.relative_path, d.chunk
      FROM
        docs_chunks_table d
      INNER JOIN
        RAW_TEXT r
        ON d.relative_path = r.relative_path
      WHERE 
        d.chunk_index = 0
    ),
    docs_category_cte AS (
      SELECT
        relative_path,
        TRIM(snowflake.cortex.CLASSIFY_TEXT (
          'Title:' || relative_path || 'Content:' || chunk, ['Bike', 'Snow']
        )['label'], '"') AS category
      FROM
        unique_documents
    )
    SELECT
      *
    FROM
      docs_category_cte;

    -- Update cathegories

    update docs_chunks_table 
        SET category = docs_categories.category
        from docs_categories
        where  docs_chunks_table.relative_path = docs_categories.relative_path;

END;
$$;


Ahora podemos crear una Task que cada X minutos puede verificar si hay nuevos datos en el stream y tomar una acción. Estamos configurando el schedule a 5 minutos para que puedas seguir la ejecución, pero siéntete libre de reducir el tiempo a 1 minuto si es necesario. Considera qué sería mejor para tu aplicación y con qué frecuencia se actualizan los nuevos documentos.

Definimos:
  - Dónde: Esto se ejecutará usando el warehouse **COMPUTE_WH**. Por favor cambia al nombre de tu propio Warehouse.
  - Cuándo: Verificar cada 5 minutos, y ejecutar en caso de que haya nuevos registros en el stream delete_docs_stream (también podríamos usar el otro stream)
  - Qué hacer: llamar al stored procedure insert_delete_docs_sp()

Ejecuta este código en tu worksheet de Snowflake para crear la task:


In [ ]:
create or replace task insert_delete_docs_task
    warehouse = COMPUTE_WH
    schedule = '5 minute'
    when system$stream_has_data('delete_docs_stream')
as
    call insert_delete_docs_sp();


alter task  insert_delete_docs_task resume;
